<a href="https://colab.research.google.com/github/DenisKai7/GroUp/blob/ml-training/GroUp_stunting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dataset dari github

In [1]:
!git clone https://github.com/DenisKai7/GroUp.git

Cloning into 'GroUp'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 66 (delta 14), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 90.62 KiB | 1.65 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
%cd GroUp
!git checkout dataset

/content/GroUp
Branch 'dataset' set up to track remote branch 'dataset' from 'origin'.
Switched to a new branch 'dataset'


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00


Import Library yang dibutuhkan

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

**Preprocessing**

load dataset

In [ ]:
#Load Dataset
data_path = '/content/GroUp/Stunting_Dataset.csv'
nutrition_data_path = '/content/GroUp/nutrition.csv'
data = pd.read_csv(data_path)
nutrition_data = pd.read_csv(nutrition_data_path)

#Preprocessing Data
class DataPreprocessor:
    def init(self, data):
        self.data = data
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()

    def preprocess(self):
        # Handle missing values
        self._handle_missing_values()
        # Feature engineering
        self._feature_engineering()
        return self._prepare_model_data()

    def _handle_missing_values(self):
        numeric_columns = self.data.select_dtypes(include=[np.number]).columns
        self.data[numeric_columns] = self.data[numeric_columns].fillna(
            self.data[numeric_columns].median()
        )

    def _feature_engineering(self):
        self.data['BMI'] = self.data['Body Weight'] / ((self.data['Body Length'] / 100) ** 2)
        self.data['Gender_Encoded'] = self.label_encoder.fit_transform(self.data['Gender'])

    def _prepare_model_data(self):
        features = ['Age', 'Birth Weight', 'Birth Length', 'Body Weight',
                    'Body Length', 'BMI', 'Gender_Encoded']
        X = self.data[features]
        y = self.label_encoder.fit_transform(self.data['Stunting'])

        X_scaled = self.scaler.fit_transform(X)
        return X_scaled, y

preprocessor = DataPreprocessor(data)
X, y = preprocessor.preprocess()